In [44]:
import tensorflow as tf
print(tf.__version__)
#import torchaudio
import sys,os
import pandas as pd
import numpy as np
from numpy.random import randint
import matplotlib.pyplot as plt
import librosa
import librosa.display
import re

from sklearn.preprocessing import LabelEncoder
#from keras.utils import to_categorical

# models
from sklearn import linear_model, naive_bayes, neighbors
#from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from keras.models import Sequential
#from keras.optimizers import Adam
#from keras.utils import np_utils
from sklearn import metrics 
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D


2.6.2


In [45]:
from scipy import stats

# Libraries for the evaluation
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
%matplotlib inline

In [80]:
labels = pd.DataFrame(columns=['speaker_id1', 'gender'])

f = open("E:/Libri/LibriSpeech/SPEAKERS.txt", "r", encoding="UTF8").readlines()
i = 0
for idx, line in enumerate(f):
    if idx > 11:
        parsed = re.split('\s+',line)
        if parsed[4] == 'train-clean-100':
            labels.loc[i] = parsed[0], parsed[2] # speaker_id and label (M/F)
            i += 1   

In [93]:
labels[180:220]

,speaker_id1,gender
180,6000,F
181,6019,M
182,6064,F
183,6078,F
184,6081,M
185,6147,F
186,6181,M
187,6209,M
188,6272,F
189,6367,M


In [95]:
root = "E:/Libri/LibriSpeech/train-clean-100/"
df = pd.DataFrame(columns=['gender'])                 
file_path = []

i = 0

# loading the features in the first dataframe
for path, subdirs, files in os.walk(root):
    for name in files:
        gender = path.split('/')[4]
        if name.endswith(".flac"):
            file_path.append(path + '/' + name)
            df.loc[i] = [gender]
            i += 1

In [126]:
df = pd.concat([pd.DataFrame(file_path, columns = ['path']),df],axis=1)
df

,path,path,gender
0,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
1,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
2,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
3,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
4,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
...,...,...,...
28533,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1
28534,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1
28535,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1
28536,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1


In [97]:
df["gender"].replace({"19": 0, "26": 1, "27": 1,"32": 0,"39": 0, "40": 0, "60": 1, "78": 1,"83": 0, "87": 0,"89": 0,"103": 0,"118": 1, "125": 0, "150": 0, "163": 1, "196": 1, "198":0, "200":0, "201": 1, "211":0, "226":0, "229": 1, "233": 1, "248":0, "250":0, "254":1, "289":0, "298":0, "302":0, "307":1, "311":1,
                      "322":0, "328":0, "332":1, "374":1, "403":0, "405":1, "412":1, "426":0, "441":0, "445":1, "446":1, "458":1, "460":1, "481":1,"587":0, "625":1, "669":0, "696":0, "730":0, "831":1, "839":1, "887":0, "909":1, "911":1, "1034":1, "1040":1, "1069":0, "1081":1, "1088":0, "1098":0, "1116":0, "1183":0, "1235":1, "1246":0, "1263":0, "1334":1, "1355":1, 
                      "1363":0, "1447":0, "1455":1, "1502":0, "1553":0, "1578":0, "1594":1, "1624":1, "1723":1, "1737":0, "1743":1, "1841":0, "1867":1, "1898":0, "1926":0, "1963":0, "1970":0, "1992":0, "2002":1, "2007":0, "2092":0, "2136":1, "2159":1, "2182":0, "2196":0, "2289":1, "2384":1, "2391":0, "2416":0, "2436":1, "2514":1,"2518":1, "2691":0, "2764": 0, "2817":0, "2836":0, "2843":1, "2893":1, "2910":0, 
                      "2911":1, "2952":1, "2989":0, "3112":0, "3168":1, "3214":1, "3235":0, "3240":1, "3242":1, "3259":0, "3374":1, "3436":1,
"3440": 0, "3486":1, "3526":0, "3607":1, "3664":1, "3699":1, "3723":1, "3807":1, "3830":1, "3857":1, "3879":0, "3947":0, "3982":0, "3983":0, "4014":1, "4018":1, "4051":0, "4088":0, "4137":0,
"4160": 0, "4195":0, "4214":0, "4267":1, "4297":0, "4340":0, "4362":0, "4397":1, "4406":1, "4441":1, "4481":0, "4640":0, "4680":0, "4788":1, "4813":1, "4830": 1, "4853": 0, "4859":1, "4898":1,
"5022": 0, "5049":1, "5104":1, "5163":0, "5192":1, "5322":1, "5339":0, "5390":1, "5393":0, "5456":1, "5463":1, "5514":0, "5561":0, "5652":0, "5678":1, "5688":0, "5703":1, "5750":1, 
"5778": 0, "5789":0, "5808":1, "5867":0, "6000":0, "6019":1, "6064":0, "6078":0, "6081":1, "6147":0, "6181":1, "6209":1, "6272":0, "6367":1, "6385":0, "6415":0, "6437":1, "6454":1, 
"6476": 0, "6529":1, "6531":0, "6563":1, "6818":0, "6836":1, "6848":1, "6880":1,"6925":1, "7059": 0, "7067":1, "7078":0, "7113":0, "7148":0, "7178":0, "7190":1, "7226":1, "7264":1, "7278":1, 
"7302":0, "7312":1, "7367":1, "7402":1, "7447":1, "7505":1, "7511":	0, "7517":0, "7635":0, "7780":0, "7794":0, "7800":0, "7859":0, "8014":0, "8051":0, "8063":1, "8088":1, "8095":1,
"8098":1, "8108":1, "8123":0, "8226":1, "8238":0, "8312":0, "8324":	0, "8419":1, "8425":1, "8465":0, "8468":0, "8580":1, "8609":1, "8629":1, "8630":1, "8747":1, "8770":1, "8797":1, 
"8838":1, "8975":0}, inplace=True)


In [98]:
df

,path,gender
0,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
1,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
2,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
3,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
4,E:/Libri/LibriSpeech/train-clean-100/103/103-1...,0
...,...,...
28533,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1
28534,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1
28535,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1
28536,E:/Libri/LibriSpeech/train-clean-100/911/911-1...,1


In [100]:
df.to_csv('E:/Libri/path.csv')

In [72]:
mfccs_df = pd.DataFrame(columns=['mfccs_result'])
sr = 16000
counter=0
for index,path in enumerate(df.path):
    audio, sr = librosa.load(path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40) 
    mfccs_processed = np.mean(mfccs.T,axis=0)
    
    mfccs_df.loc[counter]=[mfccs_processed]
    counter=counter+1
print(len(mfccs_df))
mfccs_df.head()

28538


,mfccs_result
0,"[-379.4557, 97.02814, -36.862133, 58.32614, -1..."
1,"[-350.67844, 104.95358, -50.797516, 63.575645,..."
2,"[-338.9241, 103.86572, -51.616165, 55.54726, -..."
3,"[-346.99313, 103.2503, -46.15793, 55.58074, -1..."
4,"[-350.19342, 107.2489, -42.73486, 57.279823, -..."


In [73]:
mfccs_df_combined = pd.concat([pd.DataFrame(mfccs_df['mfccs_result'].values.tolist()),df],axis=1)
mfccs_df_combined =mfccs_df_combined.fillna(0)
# DROP PATH COLUMN FOR MODELING
mfccs_df_combined.drop(columns='path',inplace=True)
print(mfccs_df_combined.shape)
mfccs_df_combined.head()

(28538, 41)


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,gender
0,-379.455688,97.028137,-36.862133,58.326141,-14.381740,18.083685,-13.535484,-1.980425,6.867363,-1.436182,...,6.590969,4.147519,2.915069,2.506089,2.704373,0.896116,0.402769,-2.776433,-0.392300,0
1,-350.678436,104.953583,-50.797516,63.575645,-20.301023,16.125051,-18.338022,-4.701718,5.564232,-3.718176,...,5.044738,4.084303,2.290769,1.581125,2.657886,1.554843,2.869262,-0.345722,1.761455,0
2,-338.924103,103.865723,-51.616165,55.547260,-14.175528,12.698159,-15.529408,-6.788486,6.311726,-1.932351,...,5.090059,3.858420,1.970551,1.166767,4.533372,2.390000,2.891569,-0.540090,-0.186476,0
3,-346.993134,103.250298,-46.157928,55.580742,-14.027334,6.038419,-17.537346,-1.607846,10.737667,-4.196127,...,3.346047,2.461340,2.484034,3.027887,4.524765,2.431338,2.143259,-1.152794,-0.398699,0
4,-350.193420,107.248901,-42.734859,57.279823,-20.294600,7.000971,-20.205288,0.712615,8.332971,-5.744866,...,4.356198,5.268185,3.117359,2.990666,4.309646,1.087925,3.075779,-1.321528,-0.485248,0


In [74]:
mfccs_df_combined.to_csv('E:/Libri/mfcc_train.csv')

In [75]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler as sd
from sklearn.model_selection import train_test_split
model = Sequential()

In [101]:
train_data=pd.read_csv('E:/Libri/mfcc_train.csv')
test_data=pd.read_csv('E:/Libri/mfcc_test.csv')

In [102]:
train_data.head()
#test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,gender
0,-379.455688,97.028137,-36.862133,58.326141,-14.381740,18.083685,-13.535484,-1.980425,6.867363,-1.436182,...,6.590969,4.147519,2.915069,2.506089,2.704373,0.896116,0.402769,-2.776433,-0.392300,0
1,-350.678436,104.953583,-50.797516,63.575645,-20.301023,16.125052,-18.338022,-4.701718,5.564232,-3.718176,...,5.044738,4.084303,2.290769,1.581125,2.657886,1.554843,2.869262,-0.345722,1.761455,0
2,-338.924103,103.865723,-51.616165,55.547260,-14.175528,12.698159,-15.529408,-6.788486,6.311726,-1.932351,...,5.090059,3.858420,1.970551,1.166767,4.533372,2.390000,2.891569,-0.540090,-0.186476,0
3,-346.993133,103.250298,-46.157928,55.580742,-14.027334,6.038419,-17.537346,-1.607846,10.737667,-4.196127,...,3.346047,2.461340,2.484034,3.027887,4.524765,2.431338,2.143259,-1.152794,-0.398699,0
4,-350.193420,107.248901,-42.734859,57.279823,-20.294600,7.000971,-20.205288,0.712615,8.332971,-5.744866,...,4.356198,5.268185,3.117359,2.990666,4.309646,1.087925,3.075779,-1.321528,-0.485248,0


In [105]:
train_data.gender.value_counts()
#test_data.gender.value_counts()

0    14341
1    14197
Name: gender, dtype: int64

In [106]:
x_train = train_data.iloc[:, 0:-1]
y_train = train_data.iloc[:, -1]

In [107]:
x_test = test_data.iloc[:, 0:-1]
y_test = test_data.iloc[:, -1]

In [108]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((28538, 40), (28538,), (2620, 40), (2620,))

In [109]:
x_train1 = np.array(x_train)
x_test1 = np.array(x_test)
y_train1 = np.array(y_train)
y_test1 = np.array(y_test)

In [110]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [111]:
x_train = x_train[:,:,np.newaxis]
x_test = x_test[:,:,np.newaxis]

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train)) 
y_test = to_categorical(le.fit_transform(y_test)) 

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((28538, 40, 1), (28538, 2), (2620, 40, 1), (2620, 2))

In [112]:
y_train.shape[1]

2

In [113]:
import tensorflow as tf
from tensorflow import keras 
from keras.layers import Layer, Input
from keras import Sequential, Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv1D,Activation,GRU,Bidirectional,BatchNormalization,Attention
from tensorflow.keras.layers import MaxPooling1D,GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import Layer, Input, Conv1D, Bidirectional, LSTM, Dense, Dropout, BatchNormalization, GlobalMaxPooling1D, Flatten
import tensorflow as tf 

sequence_input= Input(shape=(40,1), dtype='float32')
a=Conv1D(filters=32, kernel_size=7, padding='same', activation='relu')(sequence_input)
a=BatchNormalization()(a)
a=MaxPooling1D(pool_size=2)(a)
#x=Dropout(0.3)(x)
c=Conv1D(filters=64, kernel_size=9, padding='same', activation='relu')(a)
c=BatchNormalization()(c)
c=MaxPooling1D(pool_size=2)(c)
c=Conv1D(filters=64, kernel_size=7, padding='same', activation='relu')(c)
c=BatchNormalization()(c)
c=MaxPooling1D(pool_size=2)(c)
#c=Conv1D(filters=128, kernel_size=5, padding='same', activation='relu')(c)
#c=BatchNormalization()(c)
#c=GlobalAveragePooling1D()(c)
#c=Dropout(0.3)(c)
d=Bidirectional(GRU(units=32,return_sequences=True))(c)
d=Dropout(0.2)(d)
d=Bidirectional(GRU(units=16,return_sequences=True))(d)

d=GlobalAveragePooling1D()(d)
e = Dense(16, activation='relu')(d)
outp=Dense(units=2, activation='softmax')(e)
model= Model(sequence_input,outp) 
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 40, 1)]           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 40, 32)            256       
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 32)            128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 20, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 64)            18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 64)            0     

In [114]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 10)

In [115]:
epochs = 50
learning_rate = 0.0001
optimizer=keras.optimizers.Adam(lr=0.0001, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

C:\Users\CET_Pc\Anaconda3\envs\tensor\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [116]:
history = model.fit(x_train, y_train,batch_size=32,epochs=epochs,verbose=1,shuffle=True,validation_data=(x_test, y_test),callbacks=[es])        


Epoch 1/50
892/892 [==============================] - 20s 13ms/step - loss: 0.2205 - accuracy: 0.9282 - val_loss: 0.3266 - val_accuracy: 0.8760
Epoch 2/50
892/892 [==============================] - 10s 11ms/step - loss: 0.0842 - accuracy: 0.9732 - val_loss: 0.2844 - val_accuracy: 0.8954
Epoch 3/50
892/892 [==============================] - 10s 12ms/step - loss: 0.0556 - accuracy: 0.9823 - val_loss: 0.2372 - val_accuracy: 0.9183
Epoch 4/50
892/892 [==============================] - 10s 11ms/step - loss: 0.0429 - accuracy: 0.9861 - val_loss: 0.3743 - val_accuracy: 0.8882
Epoch 5/50
892/892 [==============================] - 10s 11ms/step - loss: 0.0336 - accuracy: 0.9893 - val_loss: 0.3527 - val_accuracy: 0.8897
Epoch 6/50
892/892 [==============================] - 10s 12ms/step - loss: 0.0287 - accuracy: 0.9911 - val_loss: 0.3389 - val_accuracy: 0.9053
Epoch 7/50
892/892 [==============================] - 10s 12ms/step - loss: 0.0251 - accuracy: 0.9919 - val_loss: 0.3710 - val_accuracy:

In [120]:
score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])  

Train loss: 0.00572972884401679
Train accuracy: 0.9986333847045898


In [117]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])  

Test loss: 0.3539958596229553
Test accuracy: 0.9167938828468323


In [125]:
y_pred=model.predict(x_test)
from sklearn.metrics import classification_report
y_pred = (y_pred > 0.5)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1389
           1       0.92      0.90      0.91      1231

   micro avg       0.92      0.92      0.92      2620
   macro avg       0.92      0.92      0.92      2620
weighted avg       0.92      0.92      0.92      2620
 samples avg       0.92      0.92      0.92      2620

